In [1]:
import pandas as pd

#features
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

#models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble  import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

#CV
from sklearn.model_selection import cross_validate


In [2]:
original_df = pd.read_csv("C:/Users/mcacs/Downloads/fakeWhatsApp.BR_2018.csv")

C:\Users\mcacs\anaconda3\envs\sidi\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
original_df = original_df[original_df['midia']==0]
original_df = original_df[original_df['viral']==1]
original_df.shape

(20872, 19)

In [4]:
original_df.head()

,id,date,hour,timestamp,ddi,country,country_iso3,ddd,state,group,midia,url,characters,words,types,viral,shares,text,misinformation
1,3631133147603888180,01/08/18,13:24,2018-08-01 13:24:00,55,BRASIL,BRA,17,São Paulo,2018_1,0,0,58,9,9,1,2,O Bolsonaro tem que estar preparado pra respon...,-1
15,3631133147603888180,10/08/18,14:31,2018-08-10 14:31:00,55,BRASIL,BRA,17,São Paulo,2018_1,0,0,99,18,18,1,2,"No dia 07 de Outubro, quando for votar, não es...",1
26,-4391661641377612003,11/08/18,19:28,2018-08-11 19:28:00,55,BRASIL,BRA,13,São Paulo,2018_1,0,0,114,17,17,1,6,Devido a aborrecimentos nas varias redes socia...,0
43,3631133147603888180,15/08/18,19:23,2018-08-15 19:23:00,55,BRASIL,BRA,17,São Paulo,2018_1,0,1,330,47,40,1,7,EU GOSTARIA DE PEDIR A TODOS DO GRUPOS ENTREM ...,0
103,-4391661641377612003,22/08/18,22:11,2018-08-22 22:11:00,55,BRASIL,BRA,13,São Paulo,2018_1,0,0,226,39,34,1,8,Como são as coisas.\nChefe do jacaré aparece n...,1


In [5]:
original_df = original_df.drop_duplicates(subset=['text'])
original_df.shape

(5284, 19)

In [6]:
text_df = original_df[["text","misinformation"]]
text_df

,text,misinformation
1,O Bolsonaro tem que estar preparado pra respon...,-1
15,"No dia 07 de Outubro, quando for votar, não es...",1
26,Devido a aborrecimentos nas varias redes socia...,0
43,EU GOSTARIA DE PEDIR A TODOS DO GRUPOS ENTREM ...,0
103,Como são as coisas.\nChefe do jacaré aparece n...,1
...,...,...
280574,Amoedo - Itaú - Metacapitalistas (George Soros...,1
280875,*BEM GALERA... O VIDEO DA PSICOLOGIA REVERSA P...,0
280893,DIVULGUEM ESSE VIDEO PARA QUE TODOS SAIBAM QUE...,1
281281,https://www.oantagonista.com/brasil/exclusivo-...,0


In [7]:
text_df.dtypes

text              object
misinformation     int64
dtype: object

In [8]:
text_df["misinformation"].value_counts()

 0    2547
 1    2041
-1     696
Name: misinformation, dtype: int64

In [9]:
text_df = text_df[text_df["misinformation"]>-1]

In [10]:
pip install clean-text[gpl]==0.4.0

Note: you may need to restart the kernel to use updated packages.


In [22]:
from string import punctuation

list_text = list(text_df['text'])

from cleantext import clean
for (index, text) in enumerate(list_text):
    text = clean(text, no_emoji=True)
    list_text[index] = text
text_df.drop('text', axis=1, inplace=True)
text_df['text'] = list_text
text_df.iloc[10]['text']

C:\Users\mcacs\AppData\Local\Temp/ipykernel_13140/4139207567.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text_df['text'] = list_text


'na barra da tijuca no rio, logo apos a entrevista de bolsonaro no jornal nacional\n.'

In [23]:
text_df.to_csv('C:/Users/mcacs/Downloads/sememoji.csv')

In [13]:
X_train, X_test, y_train, y_test = train_test_split(text_df['text'], text_df['misinformation'], test_size=0.2, 
                                                                    stratify = text_df['misinformation'], random_state=42)
y_train.shape

(3670,)

In [14]:
text = text_df['text']
text

15        no dia 07 de outubro, quando for votar, nao es...
26        devido a aborrecimentos nas varias redes socia...
43        eu gostaria de pedir a todos do grupos entrem ...
103       como sao as coisas.\nchefe do jacare aparece n...
112       a ratoeira funcionou mais a ratazana aguentou ...
                                ...                        
280574    amoedo - itau - metacapitalistas (george soros...
280875    *bem galera... o video da psicologia reversa p...
280893    divulguem esse video para que todos saibam que...
281281    https://www.oantagonista.com/brasil/exclusivo-...
281835    *#bolsonaro2018*\n*1*- o foco e trocar informa...
Name: text, Length: 4588, dtype: object

In [15]:
text=text.to_frame()

In [16]:
text.reset_index(inplace=True)

In [17]:
text=text['text']

In [18]:
list_text=list(text)